In [1]:
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

import json

import os
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem, SoftwareEngine, HardwareType, SoftwareType, Popularity

import sys

In [2]:
def get_soup(url,parser):
    """
    Download the html via request library,
    Useing User Agent to avoid anti-scraer detiction 
    Using beautifulsoup to parse content
    
    Arguments
    url for the target site 
    parser to be used to parse data('lxml','html','html5lib')
    
    return soup of html content
    
    """
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=10000)

        # Get list of user agents.
    user_agents = user_agent_rotator.get_user_agents()

        # Get Random User Agent String.
    user_agent = user_agent_rotator.get_random_user_agent()
    headers = {'User-agent':user_agent}
    if not os.path.exists('response'):
        os.makedirs('response')
    r = req.get(url, headers = headers)
    with open ('response/response',mode = 'w', encoding = 'UTF-8') as file:
        file.write(r.text)
    with open ('response/response',mode = 'r', encoding = 'UTF-8') as file:
        soup = bs(file,parser)
    return(soup)

In [3]:
def get_categories(url):
    soup = get_soup(url,'lxml')
    category_list = soup.findAll('li', class_="greyText")
    cat_dic = {}
    for i in range(len(category_list)):
        category = category_list[i].text.split()[0]
        link = category_list[i].a['href']
        cat_dic[category] = main_link+link
    return cat_dic

In [4]:
def get_data(soup):
    quote_list = soup.findAll('div', attrs={'class':'quote'})
    d={}
    for q in quote_list:
        try:
            author = q.find('div',  attrs={'class':'quoteDetails'}).span.text.split('\n')[1].strip()
            text = q.find('div',  attrs={'class':'quoteDetails'}).find('div',  attrs={'class':'quoteText'}).text.split('\n')[1].strip().replace('”','').replace('“','')
        except:
            continue
        if author and text:
            d ['quot'] = text
            d ['author'] = author
            d['category'] = category

        with open("data.txt", "a") as outfile:
            json.dump(d, outfile)
            outfile.write('\n')

In [5]:
def get_dataframe(file_name):
    df_list = []
    with open (file_name,"r") as j_file:
        for l in j_file:
            dic = {}
            data=json.loads(l)
            try:
                dic['quot'] = data['quot']
                dic['author'] = data['author']
                dic['category'] = data['category']
                df_list.append(dic)
            except:
                continue
    df = pd.DataFrame(df_list,columns = ['quot','author','category'])
    return df

In [6]:
main_link = 'https://www.goodreads.com'
url = main_link + '/quotes'
categories = get_categories(url)

In [7]:
scraped_categories = []
scraped_page = []

In [12]:
for k ,v in categories.items():
    if k in scraped_categories:
        print('\n',k)
        for i in range(101):
            sys.stdout.write('\r')
            sys.stdout.write("[%-10s] %d%%" % ('*'*i, 1*i))
            sys.stdout.flush()
        continue
        
    url = v + '?page='
    category = k
    print('\n', k)
    
    for i in range(1,101):
        sys.stdout.write('\r')
        sys.stdout.write("[%-10s] %d%%" % ('*'*i, 1*i))
        sys.stdout.flush()
        if i in scraped_page:
            continue
        page_url = url+str(i)
        soup = get_soup(page_url, 'lxml')
        get_data(soup)
        scraped_page.append(i)
    scraped_page = []
        
    scraped_categories.append(k)


 Love
[****************************************************************************************************] 100%
 Life
[****************************************************************************************************] 100%
 Inspirational
[****************************************************************************************************] 100%
 Humor
[****************************************************************************************************] 100%
 Philosophy
[****************************************************************************************************] 100%
 God
[****************************************************************************************************] 100%
 Truth
[****************************************************************************************************] 100%
 Wisdom
[****************************************************************************************************] 100%
 Poetry
[************************************************************

In [14]:
df = get_dataframe('data.txt')
df

,quot,author,category
0,"I love you as certain dark things are loved, s...","Pablo Neruda,",Love
1,It is an absolute human certainty that no one ...,"John Joseph Powell,",Love
2,If I have learned anything in this long life o...,"Kristin Hannah,",Love
3,your handtouching mine.this is howgalaxiescoll...,Sanober Khan,Love
4,I choose to love you in silence…For in silence...,Rumi,Love
...,...,...,...
79613,The desire for human is good but do not forget...,Lailah Gifty Akita,Motivation
79614,Don't dim your light for someone who asks for ...,Govannie de Sadeleer,Motivation
79615,"I alle forhold findes meddelagtighed, selv i b...","Peter Schellenbaum,",Motivation
79616,Package your gift and talent in such a way tha...,Dr. Lucas D. Shallua,Motivation


In [ ]:
df.to_csv('output file.csv')